# Setup Variables


In [1]:
aws_profile_name = "dev"
aws_region = "us-east-1"

In [2]:
from langchain.globals import set_verbose, set_debug

set_debug(False)
set_verbose(False)

# Setup AWS Bedrock Model


In [3]:
import boto3
from langchain_aws import ChatBedrockConverse

boto3.setup_default_session(profile_name=aws_profile_name)
BEDROCK_CLIENT = boto3.client("bedrock-runtime", aws_region)
model = ChatBedrockConverse(
    client=BEDROCK_CLIENT,
    model="anthropic.claude-3-5-sonnet-20240620-v1:0",
    max_tokens=1000,
    temperature=0.2,
    top_p=0.9,
)

# Define Our Tools


## Get Teammates By Name

Go to our "api" to get all teammates that have similar names. In this case, our API is actually just a JSON file we read in at runtime.


In [4]:
from langchain_core.tools import tool
from pydantic import BaseModel, TypeAdapter


class Teammate(BaseModel):
    id: int
    name: str
    favorite_color: str


@tool(parse_docstring=True)
def get_teammates_by_name(search_str: str) -> list[Teammate]:
    """Call to an API to get teammate details searching by their name.

    Args:
        search_str: The search string used to look for a teammate.
            This can be a partial string and will return all teammates
            who contain this search string as a substring, case insensitive.

    Returns:
        list[Teammate]: A list of any teammates with names containing the search string.
    """
    with open("data/teammates.json", "r") as file:
        teammates = TypeAdapter(list[Teammate]).validate_json(file.read())
    return [
        teammate
        for teammate in teammates
        if search_str.lower() in teammate.name.lower()
    ]

## Consolidate All Our Tools


In [5]:
from langchain_core.tools import StructuredTool
from typing import cast

tools = cast(list[StructuredTool], [get_teammates_by_name])

# Assembling our Graph


Overall we will want our final graph to do these things:

1. Pass in the user's questions
2. Continue to call tools as long as the LLM thinks it will be helpful
3. Respond with the LLM output once it is done calling tools


## First, State Definition and Some Helper Methods

This defines what state we will be passing around our graph every time we need to execute or make a decision of where to go next.

In this case, we simply want to use the built in `MessagesState`, so nothing custom is needed.


## Now the Exectuion Methods. These will make our graph `nodes`.

All of these methods will receive our graph state and return and object that will be merged with our graph state. This can just be our modified full state again if that is easier.

If returning an object with an list, it adds the new values to the list instead of replacing.


In [6]:
from langgraph.graph import StateGraph, MessagesState
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode


def call_model(state: MessagesState):
    print("Calling Model")
    print(state)
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
            Use tools when needed instead of asking permission.
            Don't reference the tool use in your response content.
        """,
            ),
            *state["messages"],
        ]
    )
    chain = prompt | model.bind_tools(tools)
    response = chain.invoke({})
    return {"messages": [response]}


tool_node = ToolNode(tools)

## Now Decision Making/Routing Methods. These will make our graph `edges`.

All of these methods will receive our graph state object and return a string that represents the next node that should be executed.

Routing methods are only needed when there is a `conditional` routing event. Some events you will see later on are simply hard coded since they always follow each other.


In [7]:
from langgraph.graph import END, MessagesState
from typing import Literal, cast
import json
from langchain_core.messages import AIMessage


def should_call_tool_or_end(state: MessagesState):
    last_message = cast(AIMessage, state["messages"][-1])
    if last_message.tool_calls:
        print(f"Calling tool(s): {json.dumps(last_message.tool_calls, indent=2)}")
        return tool_node.name
    return END

## Now we put it all together!


In [8]:
graph = StateGraph(MessagesState)

# definitions
graph.add_node(call_model)
graph.add_node(tool_node)

# navigation
graph.set_entry_point(call_model.__name__)
graph.add_conditional_edges(
    call_model.__name__,
    should_call_tool_or_end,
)
graph.add_edge(tool_node.name, call_model.__name__)

agent = graph.compile()
agent.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
       +------------+         
       | call_model |         
       +------------+         
         *         .          
       **           ..        
      *               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


# Now that we have our agent assembled, we just need to seed it with some messages and get back results!


In [9]:
from langchain_core.messages import HumanMessage

output = agent.invoke(
    {"messages": [HumanMessage("Do Matt and Grey have the same favorite color?")]}
)

Calling Model
{'messages': [HumanMessage(content='Do Matt and Grey have the same favorite color?', additional_kwargs={}, response_metadata={}, id='35c587bb-0cc5-43cd-8578-8e11c6b05aa2')]}
Calling tool(s): [
  {
    "name": "get_teammates_by_name",
    "args": {
      "search_str": "Matt"
    },
    "id": "tooluse_5NaVLwieQ9eOCmx-Nr1Nlw",
    "type": "tool_call"
  }
]
Calling Model
{'messages': [HumanMessage(content='Do Matt and Grey have the same favorite color?', additional_kwargs={}, response_metadata={}, id='35c587bb-0cc5-43cd-8578-8e11c6b05aa2'), AIMessage(content=[{'type': 'text', 'text': "To answer this question, I'll need to look up information about Matt and Grey using the available tool. Let me do that for you."}, {'type': 'tool_use', 'name': 'get_teammates_by_name', 'input': {'search_str': 'Matt'}, 'id': 'tooluse_5NaVLwieQ9eOCmx-Nr1Nlw'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '3e1a9a8d-7e5d-4475-8c36-f241a91a894f', 'HTTPStatusCode': 200, 

## The Final Answer


In [10]:
print(output["messages"][-1].content)

Based on the information I've gathered:

1. Matt Vincent's favorite color is green.
2. Grey Lovelace's favorite color is purple.

To answer your question: No, Matt and Grey do not have the same favorite color. Matt's favorite color is green, while Grey's favorite color is purple.


## Message Details


In [11]:
import json

for message in output["messages"]:
    print(json.dumps(message, indent=2, default=lambda x: x.dict()))

{
  "content": "Do Matt and Grey have the same favorite color?",
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "human",
  "name": null,
  "id": "35c587bb-0cc5-43cd-8578-8e11c6b05aa2",
  "example": false
}
{
  "content": [
    {
      "type": "text",
      "text": "To answer this question, I'll need to look up information about Matt and Grey using the available tool. Let me do that for you."
    },
    {
      "type": "tool_use",
      "name": "get_teammates_by_name",
      "input": {
        "search_str": "Matt"
      },
      "id": "tooluse_5NaVLwieQ9eOCmx-Nr1Nlw"
    }
  ],
  "additional_kwargs": {},
  "response_metadata": {
    "ResponseMetadata": {
      "RequestId": "3e1a9a8d-7e5d-4475-8c36-f241a91a894f",
      "HTTPStatusCode": 200,
      "HTTPHeaders": {
        "date": "Thu, 24 Oct 2024 18:21:02 GMT",
        "content-type": "application/json",
        "content-length": "431",
        "connection": "keep-alive",
        "x-amzn-requestid": "3e1a9a8d-7e5d-4475-